## 加载环境变量

In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)


## Zero-Shot 零样本学习
Zero-Shot 指的是在提示词中不提供任何示例，直接向模型提出任务。模型必须仅凭其在训练过程中学到的知识和指令，来完成任务。
优点：简单、快速，不需要准备示例数据。
缺点：对于复杂的、模糊的或需要特定格式的任务，效果可能不佳。

In [4]:
response = llm.invoke("What is 2 😊 9?")
print(response)

content='It looks like you\'re asking about the operation "2 😊 9". However, the meaning of the emoji "😊" in this context isn\'t a standard mathematical operation. \n\nIn mathematics, common operations between two numbers include:\n- Addition: 2 + 9 = 11\n- Subtraction: 2 - 9 = -7\n- Multiplication: 2 × 9 = 18\n- Division: 2 ÷ 9 ≈ 0.222...\n\nIf "😊" represents a specific operation or function in a particular context (e.g., a custom operation in a game, puzzle, or programming language), you would need to define what "😊" stands for. \n\nCould you clarify what "😊" represents in this case? For now, here are the results of standard operations:\n- 2 + 9 = 11\n- 2 - 9 = -7\n- 2 × 9 = 18\n- 2 ÷ 9 ≈ 0.222...\n- 2^9 = 512 (exponentiation)' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 225, 'prompt_tokens': 12, 'total_tokens': 237, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_p

## Few-Shot（少样本学习）
Few-Shot 指的是在提示词中提供少量（通常是几个）示例，以此来引导模型理解任务的模式或期望的输出格式。模型会通过这些示例进行“类比”和“推理”，然后完成你的任务。
优点：能显著提高模型在复杂任务上的表现，尤其是在需要特定格式或风格时。
缺点：需要准备高质量的示例数据。

In [6]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 增加示例组
examples = [
    {"input": "2 😊 2", "output": "4"},
    {"input": "2 😊 3", "output": "5"},
]

# 构造提示词模板
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# 组合示例与提示词
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
# 打印组合好后的提示词
print(few_shot_prompt.invoke({}).to_messages())

# 最终提示词
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位神奇的数学奇才"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 使用Few Shot 提示词进行提问
chain = final_prompt | llm
response = chain.invoke({"input": "What is a 2 😊 9?"})
print(response)

[HumanMessage(content='2 😊 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 😊 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})]
content="It looks like you're using the symbol 😊 as a custom operator! Based on the previous examples:\n\n- 2 😊 2 = 4  \n- 2 😊 3 = 5  \n\nWe can deduce that the operation `a 😊 b` is equivalent to **a + b**.  \n\nSo, **2 😊 9 = 2 + 9 = 11**.  \n\nFinal answer: **11** 😊" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 37, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'deepseek-v3-250324', 'system_fingerprint': None, 'id': '02175672014645184